In [38]:
# ProjectTwoDashboard.py
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
import plotly.express as px
import base64
import pandas as pd
import numpy as np
import os
import imghdr
import socket
import warnings

# infer proxy for Codio/Apporto if needed
JupyterDash.infer_jupyter_proxy_config()

# Import local AnimalShelter module (ensure animal_shelter.py is in same folder)
from animal_shelter import AnimalShelter


In [39]:
# -------------------------
# Config: DB credentials & logo path
# -------------------------
DB_USERNAME = "aacuser"
DB_PASSWORD = "Sikdar321"

# <-- YOUR PROVIDED LOGO PATH -->
DEFAULT_LOGO_PATH = "/home/codio/workspace/code_files/Grazioso Salvare Logo.png"


In [40]:
# -------------------------
# Helper: Find free port
# -------------------------
def find_free_port():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('', 0))
    addr, port = s.getsockname()
    s.close()
    return port


In [41]:
# -------------------------
# Helper: load & base64-encode a logo if available
# -------------------------
def load_logo_b64(path):
    if not path or not os.path.exists(path):
        return None, None
    kind = imghdr.what(path)
    # if imghdr can't detect but file exists, still attempt as png
    if not kind:
        # try extension fallback
        ext = os.path.splitext(path)[1].lower()
        if ext in ('.png', '.jpg', '.jpeg'):
            kind = 'png' if ext=='.png' else 'jpeg'
        else:
            return None, None
    mime = f"image/{'jpeg' if kind == 'jpg' else kind}"
    try:
        with open(path, "rb") as f:
            b64 = base64.b64encode(f.read()).decode()
            data_uri = f"data:{mime};base64,{b64}"
            return data_uri, mime
    except Exception as e:
        print("Error encoding logo:", e)
        return None, None


In [42]:
# -------------------------
# Connect to MongoDB & load data
# -------------------------
db = None
try:
    db = AnimalShelter(DB_USERNAME, DB_PASSWORD)
    records = db.read({}) or []
except Exception as e:
    warnings.warn(f"Could not connect to MongoDB or read data: {e}. Falling back to CSV if present.")
    records = []

# Fallback to CSV if no Mongo results and CSV exists
if not records and os.path.exists("aac_shelter_outcomes.csv"):
    try:
        df = pd.read_csv("aac_shelter_outcomes.csv")
        records = df.to_dict(orient='records')
    except Exception as e:
        print("CSV fallback failed:", e)
        records = []

df = pd.DataFrame.from_records(records)

# remove _id if present
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# ensure columns exist
if 'location_lat' not in df.columns:
    df['location_lat'] = np.nan
if 'location_long' not in df.columns:
    df['location_long'] = np.nan
if 'breed' not in df.columns:
    df['breed'] = df.get('breed', df.get('animal_type', 'Unknown'))
if 'age_upon_outcome_in_weeks' not in df.columns:
    df['age_upon_outcome_in_weeks'] = df.get('age_upon_outcome_in_weeks', np.nan)

master_df = df.copy()


MongoDB connection successful to database 'aac' (collection 'animals').


In [43]:
# -------------------------
# Rescue mapping
# -------------------------
RESCUE_MAP = {
    "Water Rescue": ["Labrador Retriever", "Newfoundland", "Portuguese Water Dog", "Chesapeake Bay Retriever"],
    "Mountain/Wilderness Rescue": ["German Shepherd", "Border Collie", "Australian Shepherd", "Siberian Husky"],
    "Disaster/Individual Tracking": ["Belgian Malinois", "Bloodhound", "German Shepherd", "Labrador Retriever"]
}

def query_for_rescue(rescue_type):
    if rescue_type not in RESCUE_MAP:
        return {}
    breeds = RESCUE_MAP[rescue_type]
    query = {"breed": {"$in": breeds}}
    query_age = {"age_upon_outcome_in_weeks": {"$lte": 104}}
    return {"$and": [query, query_age]}


In [44]:
# -------------------------
# Logo
# -------------------------
logo_data_uri, logo_mime = load_logo_b64(DEFAULT_LOGO_PATH)
if logo_data_uri:
    print("Logo loaded from:", DEFAULT_LOGO_PATH)
else:
    print("Logo not found or not recognized as image at:", DEFAULT_LOGO_PATH)
    # fallback to a local logo.png if exists
    if os.path.exists("logo.png"):
        logo_data_uri, logo_mime = load_logo_b64("logo.png")
        if logo_data_uri:
            print("Using logo.png from working directory.")


Logo loaded from: /home/codio/workspace/code_files/Grazioso Salvare Logo.png


In [45]:
# -------------------------
# Build Dash app layout
# -------------------------
app = JupyterDash(__name__)

breed_options = [{'label': b, 'value': b} for b in sorted(df['breed'].dropna().unique())]

header_children = []
if logo_data_uri:
    header_children.append(
        html.Div([
            html.Img(src=logo_data_uri, style={'height':'90px', 'marginRight':'20px', 'display':'inline-block', 'borderRadius':'6px'})
        ], style={'display':'inline-block', 'verticalAlign':'middle'})
    )
else:
    header_children.append(html.Div("Grazioso Salvare (logo missing)", style={'marginRight':'20px'}))

header_children.append(
    html.Div([
        html.H1("Grazioso Salvare — Search & Rescue Candidate Dashboard"),
        html.Div("Unique ID: Manoj Chaudhary — CS-340 Project Two", style={'fontWeight':'bold', 'color':'#b30000'})
    ], style={'display':'inline-block', 'verticalAlign':'middle'})
)

app.layout = html.Div([
    html.Div(header_children, style={'display':'flex', 'alignItems':'center'}),
    html.Hr(),

    html.Div([
        html.Div([
            html.Label("Select Rescue Type:"),
            dcc.RadioItems(
                id='rescue-type',
                options=[{'label': k, 'value': k} for k in list(RESCUE_MAP.keys())] + [{'label': 'Reset (All)', 'value': 'Reset'}],
                value='Reset',
                labelStyle={'display':'block'}
            )
        ], style={'width':'20%', 'padding':'10px'}),

        html.Div([
            html.Label("Filter by max age (weeks):"),
            dcc.Slider(id='max-age-weeks', min=0, max=520, step=4, value=104,
                       marks={0: '0', 104: '104 (2 yrs)', 260: '260 (5 yrs)', 520: '520'}),
            html.Div(id='age-display', style={'marginTop':'6px'})
        ], style={'width':'50%', 'padding':'10px'}),

        html.Div([
            html.Label("Breed (optional):"),
            dcc.Dropdown(
                id='breed-dropdown',
                options=breed_options,
                multi=False,
                placeholder='Select breed to further filter...'
            ),
            html.Button("Reset Filters", id='reset-button', n_clicks=0, style={'marginTop':'8px'})
        ], style={'width':'30%', 'padding':'10px'})
    ], style={'display':'flex'}),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_current=0,
        page_size=10,
        page_action='native',
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[],
        style_table={'overflowX': 'auto', 'maxHeight':'400px'},
    ),

    html.Br(),

    html.Div(style={'display':'flex', 'gap':'20px'}, children=[
        html.Div(id='graph-id', style={'flex':1}),
        html.Div(id='map-id', style={'flex':1})
    ]),

    html.Div(id='debug', style={'display':'none'})
])


In [46]:
# -------------------------
# Callbacks
# -------------------------
@app.callback(Output('age-display', 'children'),
              [Input('max-age-weeks', 'value')])
def show_age(value):
    return f"Showing dogs with age up to {value} weeks (set by slider)."

@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type', 'value'),
     Input('max-age-weeks', 'value'),
     Input('breed-dropdown', 'value'),
     Input('reset-button', 'n_clicks')],
    [State('datatable-id', 'data')]
)
def update_table(rescue_type, max_age, breed, reset_clicks, current_data):
    from dash import callback_context
    ctx = callback_context

    if rescue_type == 'Reset' or (ctx.triggered and ctx.triggered[0]['prop_id'] == 'reset-button.n_clicks'):
        d = master_df.copy()
        d = d[d['age_upon_outcome_in_weeks'].fillna(99999) <= max_age]
        return d.to_dict('records')

    if rescue_type in RESCUE_MAP:
        query = query_for_rescue(rescue_type)
    else:
        query = {}

    if breed:
        query = {"$and": [ {"breed": {"$eq": breed}}, {"age_upon_outcome_in_weeks": {"$lte": max_age}} ] }
    else:
        if query == {}:
            query = {"age_upon_outcome_in_weeks": {"$lte": max_age}}
        else:
            if '$and' in query:
                # ensure there's an age limiter (if not present)
                if not any('age_upon_outcome_in_weeks' in str(p) for p in query['$and']):
                    query['$and'].append({"age_upon_outcome_in_weeks": {"$lte": max_age}})
            else:
                query = {"$and":[ query, {"age_upon_outcome_in_weeks": {"$lte": max_age}} ]}

    try:
        results = db.read(query) if db else []
        if not results:
            return []
        df_local = pd.DataFrame.from_records(results)
        if '_id' in df_local.columns:
            df_local.drop(columns=['_id'], inplace=True)
        return df_local.to_dict('records')
    except Exception:
        # fallback to client-side
        d = master_df.copy()
        if breed:
            d = d[d['breed'] == breed]
        if rescue_type in RESCUE_MAP:
            d = d[d['breed'].isin(RESCUE_MAP[rescue_type])]
        d = d[d['age_upon_outcome_in_weeks'].fillna(99999) <= max_age]
        return d.to_dict('records')

@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'data')]
)
def update_graphs(data):
    if data is None or len(data) == 0:
        return html.Div("No data to display.", style={'padding':'20px'})
    dff = pd.DataFrame.from_dict(data)
    breed_counts = dff['breed'].fillna("Unknown").value_counts().nlargest(6).reset_index()
    breed_counts.columns = ['breed', 'count']
    fig = px.pie(breed_counts, names='breed', values='count', title='Top Breeds (by count)')
    return dcc.Graph(figure=fig)

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'selected_rows')]
)
def update_map(viewData, selected_rows):
    if viewData is None or len(viewData) == 0:
        return dl.Map(style={'width':'100%', 'height':'500px'}, center=[30.2672, -97.7431], zoom=10, children=[dl.TileLayer()])

    dff = pd.DataFrame.from_dict(viewData)
    markers = []
    max_markers = 25
    if selected_rows:
        idx = selected_rows[0]
        if idx < len(dff):
            row = dff.iloc[idx]
            lat = row.get('location_lat', np.nan)
            lon = row.get('location_long', np.nan)
            if pd.notnull(lat) and pd.notnull(lon):
                markers.append(dl.Marker(position=[lat, lon], children=[
                    dl.Tooltip(row.get('breed', 'Unknown')),
                    dl.Popup([
                        html.H4("Animal Info"),
                        html.P(f"Name: {row.get('name', 'N/A')}"),
                        html.P(f"Breed: {row.get('breed', 'N/A')}"),
                        html.P(f"Age (weeks): {row.get('age_upon_outcome_in_weeks', 'N/A')}")
                    ])
                ]))
    else:
        for _, row in dff.head(max_markers).iterrows():
            lat = row.get('location_lat', np.nan)
            lon = row.get('location_long', np.nan)
            if pd.notnull(lat) and pd.notnull(lon):
                markers.append(dl.Marker(position=[lat, lon], children=[dl.Tooltip(row.get('breed', 'Unknown'))]))

    return dl.Map(style={'width':'100%', 'height':'500px'}, center=[30.2672, -97.7431], zoom=10, children=[dl.TileLayer()] + markers)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': { 'column_id': i },
        'backgroundColor': '#D2F3FF'
    } for i in selected_columns]


In [47]:
# -------------------------
# Run server (safe port selection)
# -------------------------
def run_dash_inline():
    port = find_free_port()
    print(f"Launching dashboard on port {port} (inline).")
    app.run_server(mode='inline', debug=True, port=port)

if __name__ == '__main__':
    run_dash_inline()

Launching dashboard on port 38311 (inline).
